## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [1]:
!pip show requests

Name: requests
Version: 2.32.2
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\kjm99\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, intake, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [2]:
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: 
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\kjm99\anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


In [3]:
import requests
import bs4
from bs4 import BeautifulSoup

In [4]:
print(f'request ver {requests.__version__}' )
print('beautifulsoup {}'.format(bs4.__version__))

request ver 2.32.2
beautifulsoup 4.12.3


### 1. 뉴스 제목 추출하기

In [38]:
# IT/과학 뉴스
req_param = {
    'sid' : 105
}
url = 'https://news.naver.com/section/{sid}'.format(**req_param) #실제 네이버 과학뉴스 태그의 url 주소
print(url)
# 요청 헤더 설정 : 브라우저 정보
req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}
# requests 모듈을 사용해서 http 통신
res = requests.get(url, headers = req_header)
print(type(res)) #<class 'requests.models.Response'>
print(res.status_code)
print(res.ok)

if res.ok :
    # 소스 보기 텍스트 가져오기
    html = res.text
    #print(html)
    # 데이터 parsing 을 위해 Beautifulsoup 객체 생성
    soup = BeautifulSoup(html, "html.parser")
    print(type(soup))
    # print(soup)
    print(len(soup.select('a'))) #pdf 42page 참고
    print(len(soup.select("div.sa_text a[href*='n.news.naver.com/mnews/article']"))) # a 태그 속성을 이용해서 필터링 (댓글을 달 수 있는 링크도 같이 필터링되어버린다.)
    print(len(soup.select("div.sa_text a.sa_text_title"))) #class 이름을 이용해서 필터링
    # 실제 소스 코드는 다음과 같다.
    '''
        <div class='sa_text'>
            <a href='https://n.news.naver.com/mnews/article/092/0002340904'>
    '''
    atag_list = soup.select("div.sa_text a.sa_text_title")
    #print(type(atag_list)) > <class 'bs4.element.ResultSet'>
    for atag in atag_list:
        title = atag.text.strip() #strip() = 공백 제거 함수
        url = atag['href']
        print(f'제목: {title}\n링크: {url}') 
        print('------------------------------------------------------')

else :
    print(f'Error 발생 {res.status_code}')

https://news.naver.com/section/105
<class 'requests.models.Response'>
200
True
<class 'bs4.BeautifulSoup'>
394
88
44
제목: 서울대 연구진, 배터리 ‘열폭주’ 원인 밝혔다...억제 방법까지 제시
링크: https://n.news.naver.com/mnews/article/021/0002652464
------------------------------------------------------
제목: 엔씨소프트, 서브컬처 게임 개발사 '빅게임스튜디오' 지분 및 판권 투자
링크: https://n.news.naver.com/mnews/article/014/0005223399
------------------------------------------------------
제목: 포스텍, 교수 정착지원금 최대 10억원으로…"우수 석학 유치"
링크: https://n.news.naver.com/mnews/article/001/0014854962
------------------------------------------------------
제목: 카카오헬스케어, 대형병원과 연구 협력 네트워크 확대
링크: https://n.news.naver.com/mnews/article/001/0014855411
------------------------------------------------------
제목: 더존비즈온, 2분기 영업익 204억원…전년比 19% ↑
링크: https://n.news.naver.com/mnews/article/018/0005805289
------------------------------------------------------
제목: SK C&C, 베트남 ICT 기업과 글로벌 디지털 ESG 사업 협력
링크: https://n.news.naver.com/mnews/article/119/0002858766
------------------------

In [39]:
section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}

### 1.1 뉴스제목 추출하는 함수 선언하기

In [57]:
import requests
from bs4 import BeautifulSoup

def print_news(sid, section):    
    # 요청 Parameter
    req_param = {
    'sid': sid
    }
    url = 'https://news.naver.com/section/{sid}'.format(**req_param)
    
    print(f'======> {url} {section} 뉴스 <======')
    
    # 요청 헤더 설정 : 브라우저 정보
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    
    res = requests.get(url, headers = req_header)

    if res.ok : 
        html = res.text
        soup = BeautifulSoup(html, "html.parser") # 두번째 인자가 뭔지?
        atag_list = soup.select("div.sa_text a.sa_text_title")
        for atag in atag_list:
            title = atag.text.strip() #strip() = 공백 제거 함수
            url = atag['href']
            print(f'제목: {title}\n링크: {url}') 
            print('------------------------------------------------------')

    else :
        print(f'Error 발생 {res.status_code}')

In [58]:
print_news(103,section_dict[103])

======> https://news.naver.com/section/103 생활/문화 뉴스 <======
제목: 당분간 폭염 계속…곳곳서 강한 소나기[날씨]
링크: https://n.news.naver.com/mnews/article/055/0001178631
------------------------------------------------------
제목: '제로백 3초대'…BMW, 뉴 M4 컴페티션 쿠페·컨버터블 출시
링크: https://n.news.naver.com/mnews/article/030/0003229194
------------------------------------------------------
제목: BMW, 벤츠 제치고 7월 수입차 판매 1위
링크: https://n.news.naver.com/mnews/article/215/0001173879
------------------------------------------------------
제목: 롯데렌탈·케이카, 사업 확장 속도
링크: https://n.news.naver.com/mnews/article/015/0005017879
------------------------------------------------------
제목: 지드래곤이 예고한  공익 재단 '저스피스' 오늘 출범
링크: https://n.news.naver.com/mnews/article/079/0003924532
------------------------------------------------------
제목: '오텔로'로 돌아온 이용훈 "동양인 성악가 마음 대변하는 작품"
링크: https://n.news.naver.com/mnews/article/001/0014855229
------------------------------------------------------
제목: 다리 휘었나요? 그냥 두다간 ‘이 병’으로 이어질 수도
링크: https://n.news.naver.com/mne

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [78]:
import requests
import os

req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=798173&no=5&amp;weekday=thu'
}
img_urls = [
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg'
]

for img_url in img_urls:
    # binary 데이터 추출
    res = requests.get(img_url, headers = req_header)
    if res.ok :
        img_data = res.content # url 에서 file name을 가져옴
        # print(img_data)
        file_name = os.path.basename(img_url)
        with open(file_name, 'wb') as file: # wb = binary 데이터를 쓰겠다(write)
            print(f'writing to {file_name} ({len(img_data):,} bytes)')
            file.write(img_data)
    else:
        print(f'Error 발생 {res.status_code}')

writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg (299,319 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg (163,333 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg (182,530 bytes)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [16]:
import requests
import os

url = 'https://comic.naver.com/webtoon/detail?titleId=798173&no=5&amp;weekday=thu'
req_header = {
    'referer': url # referer 헤더가 무엇인가? : 
}
#
res = requests.get(url)
if res.ok: #200
    pass
else:
    print(res.status_code)

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

### 4. 단축 URL 생성하는 Open API 사용하기
* urllib 사용

In [ ]:
import os
import sys
import urllib.request

client_id = "V" # 개발자센터에서 발급받은 Client ID 값
client_secret = "y_" # 개발자센터에서 발급받은 Client Secret 값
encText = urllib.parse.quote("https://drive.google.com/drive/u/0/folders/1V_DK7Px5w_niEKLlWhDr_GPgIYHG3Du-")
data = "url=" + encText
url = "https://openapi.naver.com/v1/util/shorturl"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
import requests

client_id = "V" # 개발자센터에서 발급받은 Client ID 값
client_secret = "y_" # 개발자센터에서 발급받은 Client Secret 값

origin_url = "https://drive.google.com/drive/u/0/folders/1V_DK7Px5w_niEKLlWhDr_GPgIYHG3Du-"



### 5. 블로그 검색하기

In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': 'V',
    'X-Naver-Client-Secret': 'y_',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'

res = requests.get(url, params=payload, headers=headers)
items_data = res.json()['items']
#print(items_data)

items_list = list()
item_list = []
for item in items_data:
#     print(item)
    item_list.append(item['title'])
    item_list.append(item['bloggername'])
    item_list.append(item['description'])
    item_list.append(item['bloggerlink'])
    item_list.append(item['link'])

    items_list.append(item_list)
    item_list = []

print(items_list)

with open('data/nhnblog.txt','w',encoding="utf-8")as file:
    for items in items_list:
        for item in items:
            item = item + '\n'
            file.write(item)
        file.write('-'*150+'\n')